# Figure 5 - Layout

In [ ]:
import sys
sys.path.append("..")

from main import *
from visualization import *

from scipy.stats import zscore
from scipy.ndimage import gaussian_filter1d

# Loading data

In [ ]:
timeseries, correlations = [], []
for i in range(4):
    timeseries.append(np.load(f'../Results/figure5_timeseries{i+1}.npy'))
    correlations.append(np.load(f'../Results/figure5_correlations{i+1}.npy'))

corrs = np.load('../ComputeCanada/filtering/mode_correlations_h_and_sigma.npy')
grid = np.zeros((20, 20))
for i in range(grid.shape[0]):
    for j in range(grid.shape[1]):
        values = []
        for k in range(10):
            diag = np.diag(corrs[i, j, k])
            values.append(np.mean(np.abs(diag)[:20]))
        grid[i, j] = np.mean(values)

mode_similarity = []
for i in range(3):
    mode_similarity.append(np.load(f'../Results/figure5_mode_similarity{i+1}.npy'))

# Rendering figure

In [ ]:
fig = PaperFigure(figsize=(7, 5))

fig.set_tick_length(2)
fig.set_font_size(6)
fig.add_background()

# Setting up axes -------------------------------------------------------------------------------------------------

w = 1.65
pad = (7 - 4 * w) / 3
for i in range(4):
    fig.add_axes(f'timeseries{i}', (i * (w + pad), 0), w, 1)

for i in range(4):
    fig.add_axes(f'correlations{i}', (i * (w + pad), 1.5), 1, 1)

fig.add_axes('grid', (0, 3), w, w)
fig.add_axes('curves', (w + 0.5, 3 + (0.25 * w)), w, w * 0.75)

w2 = 0.8
fig.add_axes('mode_similarity1', (7 - 3 * w2, 3 + w - w2), w2, w2)
fig.add_axes('mode_similarity2', (7 - 2 * w2, 3 + w - w2), w2, w2)
fig.add_axes('mode_similarity3', (7 - 1 * w2, 3 + w - w2), w2, w2)

fig.set_line_thickness(0.6)

# Filling content --------------------------------------------------------------------------------------------------

for i in range(4):
    ax = fig.axes[f'timeseries{i}']
    ax.imshow(zscore(timeseries[i][:, :1000], axis=1), cmap='binary', vmin=-0.5, vmax=1.5, aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])

for i in range(4):
    ax = fig.axes[f'correlations{i}']
    ax.imshow(correlations[i], cmap='Reds', vmin=np.percentile(correlations[i], 5), vmax=np.percentile(correlations[i], 95), aspect='auto')
    ax.set_xticks([])
    ax.set_yticks([])
    print(np.percentile(correlations[i], 5), np.percentile(correlations[i], 95))

ax = fig.axes['grid']
ax.imshow(grid, cmap='Reds')

ax = fig.axes['curves']
ax.spines[['top', 'right']].set_visible(False)
red = plt.get_cmap('Reds')(175)[:3]
beige = plt.get_cmap('Reds')(50)[:3]
colors = color_gradient(red, beige, 20)
x = np.linspace(0, 0.1, 20, endpoint=True)
for i in range(20):
    ax.plot(x, gaussian_filter1d(grid[i, :], 1), color=colors[i])
ax.set_xticks([0, 0.05, 0.1])
ax.set_xlim([0, 0.1])

for i in range(3):
    ax = fig.axes[f'mode_similarity{i+1}']
    ax.imshow(mode_similarity[i], cmap='Reds', vmin=0, vmax=1)
    ax.set_xticks([])
    ax.set_yticks([])

fig.show()

In [ ]:
fig.save('../Figures/figure5_incomplete.svg')